In [15]:
# Integrantes:
# ------------------------------------------------------------------------------
# Aristeo García Marco Antonio
# Cruz Torres Antonio
# Miravete Casillas Francisco Leonardo
# Valle Chávez Abel
# Wong Mozqueda Arturo
# ------------------------------------------------------------------------------

import pandas as pd
import folium
import datetime as dt
import base64
import json
import altair as alt
import numpy as np

from folium.plugins import HeatMapWithTime

# Function that return a base map.
def generateBaseMap(default_location=[30.136913, 111.628497], default_zoom_start=3, default_tile='openstreetmap'):
    base_map = folium.Map(location=default_location, tiles=default_tile, control_scale=True, zoom_start=default_zoom_start)
    return base_map

# Begins sequential program.
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
# Get last date available.
last_date_column = list(df.columns.values)[-1]
df_mark = df[['Province/State', 'Country/Region', 'Lat', 'Long', last_date_column]]

df_unpivot = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat','Long'], var_name='Date', value_name='Cases').copy()

df_unpivot['DateFormat'] = df_unpivot['Date'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%y'))
lat = df_unpivot.Lat.mean()
lon = df_unpivot.Long.mean()

df_unpivot_clean = df_unpivot[df_unpivot['Cases'] > 0]

df_day_list = []
for date in df_unpivot_clean.DateFormat.sort_values().unique():
    df_day_list.append(df_unpivot_clean.loc[df_unpivot_clean.DateFormat == date, ['Lat', 'Long', 'Cases']].groupby(['Lat', 'Long']).sum().reset_index().values.tolist())

virus_map = generateBaseMap(default_location=[lat, lon], default_tile='cartodbpositron')
feature_group = folium.FeatureGroup(name='Histórico de Casos')

# Locate marks at the time of the last date.
for idx in range(0, len(df_mark)):
    registro = df.values[idx].tolist()
    y = registro[4:len(registro) - 1]
    x = np.arange(0, len(y), 1)
    source = pd.DataFrame({'Dias transcurridos': x,'Casos': y})
    chart = alt.Chart(source).mark_area(color="orange", interpolate='step-after', line=True).encode(
            x='Dias transcurridos',
            y=alt.Y(
                "Casos:Q",
                stack="normalize",
                title="Numero de casos",
                axis=alt.Axis(format="0")
            ))
    
    chart_view = chart.to_json()

    folium.Marker(location=[df_mark.Lat.values[idx], df_mark.Long.values[idx]],
            tooltip=("Click para ver histórico"),
            popup=folium.Popup(max_width=400).add_child(folium.VegaLite(chart_view, width=400, height=300))).add_to(feature_group)

HeatMapWithTime(df_day_list, radius=15, name="Evolución de casos").add_to(virus_map)
feature_group.add_to(virus_map)

folium.TileLayer('openstreetmap').add_to(virus_map)
folium.TileLayer('cartodbpositron').add_to(virus_map)
folium.TileLayer('cartodbdark_matter').add_to(virus_map)
folium.TileLayer('stamentoner').add_to(virus_map)
folium.TileLayer('stamenwatercolor').add_to(virus_map)
folium.TileLayer('stamenterrain').add_to(virus_map)
folium.LayerControl().add_to(virus_map)

# The map cannot be viewed in collaboratory, you need to export to html.
virus_map.save(outfile='coronavirus-time.html')